# vector 자료구조

In [175]:
from array import array
import math

class Vector2d :
    typecode ='d'
    
    def __init__(self, x,y) :
        self.__x = float(x)
        self.__y = float(y)
        
    # x,y를 일기 전용 속성으로 만든다.
    @property
    def x(self) :
        return self.__x
    @property
    def y(self) :
        return self.__y
    
    # __iter__ method 를 하면 Vector2d를 반복할 수 있고 이로써 언패킹이 가능하다.
    def __iter__(self) :
        return (i for i in (self.x, self.y))
    
    def __repr__(self) :
        class_name = self.__class__.__name__
        return f"{class_name}({self.x}, {self.y})"
    
    def __str__(self) :
        return str(tuple(self))
    
    def __bytes__(self) :
        return bytes([ord(self.typecode)]) + bytes(array(self.typecode, self))
    
    def __eq__(self, other) :
        return tuple(self) == tuple(other)
    
    # 유클리드 거리
    def __abs__(self) :
        return math.hypot(self.x, self.y)
    
    # abs가 0이면 False 나머지는 True
    def __bool__(self) :
        return bool(abs(self))
    
    def __complex__(self) :
        return complex(self.x, self.y)
    
    #classmethod는 첫번째 함수로 자신의 클래스를 받는다.
    @classmethod
    def frombytes(cls, octets) :
        typecode = chr(octets[0])
        # memoryview 는 구조체를 복사하지 않고 메모리를 공유할 수 있게 해준다.
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)
    
    def angle(self):
        return math.atan2(self.y, self.x)
    
    def __format__(self, fmt_spec='') :
        if fmt_spec.endswith('p') :
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else :
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)
    
    def __hash__(self) :
        # 클래스에 요소가 2개 이상이라면 각 요소의 해쉬의 배타곱이 권장된다.
        return hash(self.x) ^ hash(self.y)

In [176]:
v1 = Vector2d(3,4)
print(v1.x, v1.y)

3.0 4.0


In [177]:
x,y = v1
print(x,y)

3.0 4.0


In [178]:
v1_clone = eval(repr(v1))
v1 == v1_clone

True

In [179]:
print(v1)

(3.0, 4.0)


In [180]:
octets = bytes(v1)
print(octets)
v1.frombytes(octets)

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'


Vector2d(3.0, 4.0)

In [181]:
abs(v1)

5.0

In [182]:
bool(v1), bool(Vector2d(0,0))

(True, False)

In [183]:
print(f"{v1:0.2f}")

(3.00, 4.00)


In [184]:
print(f"{v1:p}")

<5.0, 0.9272952180016122>


In [185]:
print(f"{v1:0.2fp}")

<5.00, 0.93>


In [186]:
v1 = Vector2d(3,4)
v2 = Vector2d(3.1,4.2)
hash(v1), hash(v2)
set([v1,v2])

{Vector2d(3.0, 4.0), Vector2d(3.1, 4.2)}

In [187]:
complex(v1)

(3+4j)